**TODO**
- change photograph and initial plots to reflect the new domain

# Post-process PCGA run

## Red River Example
This is a first cut at bathymetry inversion on a section of the Red River below Acme, LA.

The reach is roughly 9 [km] long. The cross-channel distances range approximately between 100 and 300 m. 


### Reference bathymetry
Our reference bathymetry is built by interpolating cross-sections from a detailed multibeam survey conducted by the ERDC Coastal and Hydraulics Lab. The problem domain and interpolated bathymetry on the a reference mesh are shown below.

<table>
<tr><th> Red River Domain</th> <th> Survey-Generated Bathymetry</th>
<tr>
<td> <img src="./plots/inset_google_map.png" alt="Red River Domain" style="width:300px"> </td>
<td> <img src="./plots/bathymetry_true.png" alt="Red River Bathymetry" style="width:300px"> </td>
</table>

## Auxiliary Conditions
Boundary conditions for the problem were a constant discharge $Q_b= 829.4$ [m${}^3$/s] at the inflow (upper left) boundary and a constant tailwater (free-surface elevation) $z_b=4.5$ [m] at the outflow (lower right).

The initial conditions were
$$
\mathbf{v}=0 \\
\eta = z_b
$$
where $\eta$ is the free-surface elevation, $\eta = z+h$, and $\mathbf{v}$ is the depth averaged velocity.

The simulations were run for 1 hour using the Corps' `Adaptive Hydraulics (AdH)` 2d shallow water module.

**Note** We will probably need to run the simulation longer or use a better initial condition from an offline 'spin-up' for the actual data collection.

### Velocity and Depth profiles
The velocity and depth predicted by AdH on the reference bathymetry are below. 

**TODO** replace these with plots in consistent color scheme with bathymetry plot

<table>
<tr><th> Velocity </th> <th> Depth</th>
<tr>
<td> <img src="./plots/velocity_inset_true.png" alt="Velocity for Collection Period" style="width:300px"> </td>
<td> <img src="./plots/depth_inset_true.png" alt="Depth for Collection Period" style="width:300px"> </td>
</table>

Import the necessary python modules

In [ ]:
%matplotlib inline
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

import numpy as np
from scipy.io import savemat, loadmat
import red_inset_plot_utils as rp

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams['legend.loc'] = 'best'

In [ ]:
base_dir = os.getcwd()

## Read in the results

The location of the results and names of several files to be read


In [ ]:
res_dir = os.path.join(base_dir, 'results','inset_02262018','npc_100','no_vort')#os.getcwd()
results_file = 'results.mat'
soln_prefix = "shat"
varfile     = "postv.txt"
nodefile    = "meshnode.txt"
elemfile    = "triangles.txt"
truefile    = "true.txt"
obs_locfile = "observation_loc_N250_M8_J1_I10.dat"
obs_file    = "observations.dat"#"obs.txt"
sim_obs_template = 'simulobs{0:d}.txt'
true_file   = "z_Inset_02262018"

In [ ]:
os.chdir(res_dir)

Read in the results, stored in `soln_prefix*` and variance stored in var_file

In [ ]:
resfiles = [f for f in os.listdir(res_dir) if os.path.isfile(os.path.join(res_dir, f)) and f[:len(soln_prefix)] == soln_prefix]
resfile = resfiles[-1]


## set some problem information

In [ ]:
nx,ny = 1001,51

Go ahead and load in the results file to use if possible

In [ ]:
results = loadmat(results_file)

### the solution and posterior variance

In [ ]:
s_hat = results['s_hat']#np.loadtxt(os.path.join(res_dir,resfile))
postv= np.loadtxt(os.path.join(res_dir,varfile))
poststd = np.sqrt(postv)                  

### the computational domain

In [ ]:
triangles = np.loadtxt(elemfile)
meshnode = np.loadtxt(nodefile)


### observation locations and values

In [ ]:
velocity_obs_loc = np.loadtxt(obs_locfile)

obs = np.loadtxt(obs_file)
nobs = obs.shape[0]


In [ ]:
simul_obs_list = []
for iter in range(len(resfiles)):
    simul_obs_list.append(np.loadtxt(os.path.join(res_dir,sim_obs_template.format(iter+1))))
simul_obs = simul_obs_list[-1]

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_bathymetry_physical_domain(meshnode,triangles,title='True Red Inset Bathymetry and Obs')
ax.scatter(velocity_obs_loc[:, 0], velocity_obs_loc[:, 1], c='k', s=0.5, alpha=0.7, label='obs.')

## Visualization

Start by setting some visualization parameters and reshaping for logically rectangular grid.

In [ ]:
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
N = np.array([nx, ny])
m = np.prod(N)

s_true = np.loadtxt(true_file)

s_hat2d = s_hat.reshape(N[1], N[0])
s_true2d = s_true.reshape(N[1], N[0])
#bounds for scaling
minv = s_true.min()
maxv = s_true.max()

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_field_on_physical_domain(meshnode,triangles,s_true.flat,title='True Red Inset Bathymetry')
ax.scatter(velocity_obs_loc[:, 0], velocity_obs_loc[:, 1], c='k', s=0.5, alpha=0.7, label='obs.')
fig.savefig('./bathymetry_true.png', dpi=300, bbox_inches='tight', pad_inches=0.0)

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_field_on_physical_domain(meshnode,triangles,s_hat.flat,title='Estimated Red Inset Bathymetry')
fig.savefig('./bathymetry_estimate.png', dpi=300, bbox_inches='tight', pad_inches=0.0)
                                        

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
diff=s_hat.flat[:]-s_true.flat[:]
ax = rp.plot_field_on_physical_domain(meshnode,triangles,diff.flat,title='Difference in Estimated Bathymetry')
fig.savefig('./bathymetry_diff.png', dpi=300, bbox_inches='tight', pad_inches=0.0)
 

### The uncertainty estimate

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
title='Uncertainty Map (std): Red Inset Bathymetry'
ax = rp.plot_field_on_physical_domain(meshnode,triangles,poststd,title=title)
fig.savefig('./bathymetry_poststd.png', dpi=300, bbox_inches='tight', pad_inches=0.0)



In [ ]:
titles=['True Red Inset Bathymetry','Esimtated Red Inset Bathymetry']
s_list=[s_true.flat[:],s_hat.flat[:]]
fig,axes = rp.plot_field_physical_domain_comparison_1x2(meshnode,triangles,s_list,titles)
plt.savefig('./bathymetry_comparison.png', dpi = 300, bbox_inches='tight', pad_inches=0.0)

In [ ]:
title ='measured vs simulated velocity, RMSE : {0:5.3e}'.format(np.linalg.norm(obs - simul_obs)/np.sqrt(nobs))
fig = plt.figure(figsize=(10, 6.), dpi=300)
ax = rp.plot_observations_comparison(obs,simul_obs,title=title)
fig.savefig('obs.png', dpi=fig.dpi)

Look at the progress of the optimization

In [ ]:
fig = plt.figure(figsize=(10, 6.), dpi=300)
plt.semilogy(range(len(results['objvals'][0])), results['objvals'][0], 'r-*')
plt.title('obj values over iterations')
plt.axis('tight')
fig.savefig('obj.png', dpi=fig.dpi)

Setup the coordinate information and arrays for visualization

In [ ]:
soln = s_hat.flat[:]
mesh_true = np.array(meshnode)
mesh_est = np.array(meshnode)
mesh_est_u = np.array(meshnode)
mesh_est_l = np.array(meshnode)
mesh_est[:, 2] = soln[:]
mesh_est_u[:,2] = soln + 1.96*poststd[:]
mesh_est_l[:,2] = soln - 1.96*poststd[:]


### transverse mean comparison

In [ ]:
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_streamwise_section_comparison(mesh_true,mesh_est,
                                           mesh_est_u,mesh_est_l,nx,ny,0,
                                           use_transverse_average=True,
                                           title='transverse avg. with std(error)= 0.05 m/s',
                                           ylabel='$z_b$ [m]',xlabel='streamwise distance [m]')

### centerline

In [ ]:
J_plot=int(ny/2)
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_streamwise_section_comparison(mesh_true,mesh_est,
                                           mesh_est_u,mesh_est_l,nx,ny,J_plot,
                                           title='centerline with std(error)= 0.05 m/s',
                                           ylabel='$z_b$ [m]',xlabel='streamwise distance [m]')

### a section south of the centerline

In [ ]:
J_plot=20
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_streamwise_section_comparison(mesh_true,mesh_est,
                                           mesh_est_u,mesh_est_l,nx,ny,J_plot,
                                           title='{0:3.0f}% slice with std(error)= 0.05 m/s'.format(J_plot/float(ny)*100),
                                           ylabel='$z_b$ [m]',xlabel='streamwise distance [m]')

### a section north of the centerline

In [ ]:
J_plot=30
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_streamwise_section_comparison(mesh_true,mesh_est,
                                           mesh_est_u,mesh_est_l,nx,ny,J_plot,
                                           title='{0:3.0f}% slice with std(error)= 0.05 m/s'.format(J_plot/float(ny)*100),
                                           ylabel='$z_b$ [m]',xlabel='streamwise distance [m]')

### compare four cross sections


In [ ]:
J_list = [None,int(ny/2),20,30]
fig,axes = rp.plot_streamwise_section_comparison_2x2(mesh_true,mesh_est,mesh_est_u,mesh_est_l,nx,ny,J_list,
                                                     title_suffix = 'with std(error) = 0.05 m/s',
                                                     ylabel='$z_b$ [m]',xlabel='streamwise distance [m]')
plt.savefig('red_inset_alongchannel.png', bbox_inches='tight', pad_inches=0, dpi=300)

## Compare cross sections now

Choose a few cross-sections and label by relative distance down the channel

In [ ]:
I_plot=153
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_channel_crosssection_comparison(mesh_true,mesh_est,mesh_est_u,mesh_est_l,
                                             nx,ny,I_plot,title='{0:3.1f}% cross-section with std(error)= 0.05 [m/s]'.format(I_plot/float(nx)*100),
                                             ylabel='$z_b$ [m]',
                                             xlabel='cross-channel distance [m]',
                                            loc='lower right')

In [ ]:
I_plot=200
fig = plt.figure(figsize=(10,6),dpi=300)
ax = rp.plot_channel_crosssection_comparison(mesh_true,mesh_est,mesh_est_u,mesh_est_l,
                                             nx,ny,I_plot,title='{0:3.1f}% cross-section with std(error)= 0.05 [m/s]'.format(I_plot/float(nx)*100),
                                             ylabel='$z_b$ [m]',
                                             xlabel='cross-channel distance [m]',
                                             loc='lower right')

In [ ]:
I_plot=753
fig = plt.figure(figsize=(8,6),dpi=300)
ax = rp.plot_channel_crosssection_comparison(mesh_true,mesh_est,mesh_est_u,mesh_est_l,
                                             nx,ny,I_plot,title='{0:3.1f}% cross-section with std(error)= 0.05 [m/s]'.format(I_plot/float(nx)*100),
                                             ylabel='$z_b$ [m]',
                                             xlabel='cross-channel distance [m]',
                                            loc='lower right')

In [ ]:
I_list=[153,200,753]
fig,axes = rp.plot_channel_crosssection_comparison_3x1(mesh_true,mesh_est,mesh_est_u,mesh_est_l,
                                                 nx,ny,I_list,title_suffix='with std(error)= 0.05 [m/s]',
                                                 ylabel='$z_b$ [m]',xlabel='cross-channel distance [m]')
plt.savefig('red_inset_acrosschannel.png', bbox_inches='tight', pad_inches=0, dpi=300)